In [1]:
import pandas as pd
df=pd.read_csv('5dataSeH_model.csv')

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
df.shape

(186, 3991)

In [3]:
from sklearn.linear_model import LinearRegression
from testset_prediction import testset_prediction as tsp
from sequential_selection import stepwise_selection as sq
from loo import loo
from rm2 import rm2
reg=LinearRegression()

num_des=5   #Specify the number of descriptors to be selected in the model

def process(tr,trX,Xts,y,yts,seed):
    lt=[]
    ls=['r2','neg_mean_absolute_error','neg_mean_poisson_deviance','neg_mean_gamma_deviance']
    #ls=['r2']
    lf=[0,5]
    #lf=[0]
    l1,l2,l3,l4,l5,l6,l7=[],[],[],[],[],[],[]
    xl=[]
    for i in ls:
        for j in lf:
            sqs=sq(trX,y,num_des,True,True,i,j)
            try:
               a1,b1=sqs.fit_()
               xl.append(a1)
               print(str(len(xl))+' Model generated')
               reg.fit(tr[a1],y)
               r2tr=reg.score(tr[a1],y)
               cv=loo(tr[a1],y,tr)
               #print(a1)
               c,m,l=cv.cal()
               r2pr,r2pr2,RMSEP=testpred(Xts[a1],yts,reg,m)
               rm2tr,drm2tr=rm2(y,l).fit()
               ytspr=pd.DataFrame(reg.predict(Xts[a1]))
               rm2ts,drm2ts=rm2(yts,ytspr).fit()
            except ValueError:
                c=0
                r2pr=0
                rm2tr=0
                rm2ts=0         
            #print(c,m,l)
            #print(a1)
            l1.append(i) 
            l2.append(j)  
            l3.append(c)
            l4.append(r2pr)
            l6.append(rm2tr)
            l7.append(rm2ts)
            l5.append(seed)
    Dict=dict([('random_seed', l5),('score', l1),('fold', l2),('Q2LOO', l3), ('R2Pred', l4), ('rm2tr', l6), ('rm2ts', l7)])
    #print(Dict)
    table=pd.DataFrame(Dict)
    return table
    #tbname='Results_table.csv'
    #table.to_csv('Results_table.csv',index=False) 

In [4]:
from sklearn.feature_selection import VarianceThreshold
def correlation(X,cthreshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = X.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (abs(corr_matrix.iloc[i, j]) > float(cthreshold)) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in X.columns:
                   del X[colname] # deleting the column from the dataset
    return X   

def variance(X,threshold):
    sel = VarianceThreshold(threshold=(threshold* (1 - threshold)))
    sel_var=sel.fit_transform(X)
    X=X[X.columns[sel.get_support(indices=True)]]    
    return X


def pretreat(X,cthreshold,vthreshold):
    X=correlation(X,cthreshold)
    X=variance(X,vthreshold)
    return X

In [5]:
def testpred(Xts,yts,model,trav):
    ytspr=pd.DataFrame(model.predict(Xts))
    ytspr.columns=['Pred']
    tsdf=pd.concat([yts,pd.DataFrame(ytspr)],axis=1)
    tsdf.columns=['Active','Predict']
    tsdf['Aver']=trav
    tsdf['Aver2']=tsdf['Predict'].mean()
    tsdf['diff']=tsdf['Active']-tsdf['Predict']
    tsdf['diff2']=tsdf['Active']-tsdf['Aver']
    tsdf['diff3']=tsdf['Active']-tsdf['Aver2']
    r2pr=1-((tsdf['diff']**2).sum()/(tsdf['diff2']**2).sum())
    r2pr2=1-((tsdf['diff']**2).sum()/(tsdf['diff3']**2).sum())
    RMSEP=((tsdf['diff']**2).sum()/tsdf.shape[0])**0.5
    return r2pr,r2pr2,RMSEP

In [6]:
import pandas as pd
X=df.iloc[:,2:]
Xpt=pretreat(X,0.99,0.0001)

In [ ]:
Xpt.to_csv('Pretreated_0.99_0.0001_AS.csv')

In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd
Xpt2=Xpt.columns.tolist()[1:]
perc=20
nls=[]
for sp in range(2,5):
    #perc=int(secondEntryTabOne.get())
    print('Activity sorting: '+str(sp))
    pc=int(100/perc)
    #sp=int(secondEntryTabOne_x.get())
    filem=df.sort_values(df.iloc[:,1:2].columns[0],ascending=False)
    ts=filem.iloc[(sp-1)::pc, :]
    tr=filem.drop(ts.index.values)
    #tr.to_csv('tr.csv', index=False)
    #a,b= train_test_split(df,test_size=perc, random_state=i)
    #tr=pd.DataFrame(a)
    #ts=pd.DataFrame(b)
    tr=tr.reset_index().drop('index', axis=1)
    ts=ts.reset_index().drop('index', axis=1)
    trX=tr[Xpt2]
    Xts=ts[Xpt2]
    #print(trX.shape)
    y=tr.iloc[:,1:2]
    tr1=pd.concat([tr.iloc[:,0:2],trX],axis=1)
    ts1=pd.concat([ts.iloc[:,0:2],Xts],axis=1)
    #tr1.to_csv('tr.csv', index=False)
    #ts1.to_csv('ts.csv', index=False)
    yts=ts.iloc[:,1:2] 
    table=process(tr,trX,Xts,y,yts,sp)
    nls.append(table)
    fd=pd.concat(nls, axis=0)

Activity sorting: 2
1 Model generated
['SM03_EA(bo)', 'Eig12_AEA(ed)', 'CATS2D_07_DA', 'CATS2D_06_NL', 'SHED_DN', 'B04[O-O]', 'Neoplastic-50', 'MDEN-33']
2 Model generated
['TIE', 'BIC2', 'mindsCH', 'CATS2D_08_DD', 'CATS2D_05_DA', 'CATS2D_09_DA', 'B04[Cl-Cl]', 'Inflammat-50']
3 Model generated
['SM03_EA(bo)', 'mindsCH', 'CATS2D_07_DA', 'CATS2D_09_DA', 'CATS2D_06_NL', 'B04[O-O]', 'Neoplastic-50', 'MDEN-12']
4 Model generated
['ATS1m', 'SpMin8_Bh(p)', 'mindsCH', 'CATS2D_06_DL', 'CATS2D_06_AA', 'T(N..N)', 'F08[C-O]', 'Neoplastic-50']
5 Model generated
['SM03_EA(bo)', 'mindsCH', 'CATS2D_02_DD', 'CATS2D_07_DA', 'CATS2D_05_AL', 'CATS2D_06_NL', 'B04[O-O]', 'Neoplastic-50']
6 Model generated
['ATS1m', 'P_VSA_LogP_1', 'Eig12_AEA(ed)', 'MaxsssCH', 'B06[O-O]', 'F02[C-F]', 'F08[C-O]', 'Neoplastic-50']
7 Model generated
['SM03_EA(bo)', 'mindsCH', 'CATS2D_02_DD', 'CATS2D_07_DA', 'CATS2D_05_AL', 'CATS2D_06_NL', 'B04[O-O]', 'Neoplastic-50']
8 Model generated
['ATS1m', 'MATS2v', 'MATS5v', 'C-016', 'B06

In [9]:
fd['Average']=fd[['Q2LOO', 'R2Pred']].mean(axis=1)
fd.sort_values('Average', ascending=False)

,random_seed,score,fold,Q2LOO,R2Pred,rm2tr,rm2ts,Average
0,3,r2,0,0.771847,0.752340,0.684525,0.691859,0.762094
0,2,r2,0,0.710197,0.777097,0.608688,0.620888,0.743647
4,2,neg_mean_poisson_deviance,0,0.733838,0.745919,0.633782,0.558977,0.739878
6,2,neg_mean_gamma_deviance,0,0.733838,0.745919,0.633782,0.558977,0.739878
2,2,neg_mean_absolute_error,0,0.739934,0.739060,0.642019,0.592126,0.739497
6,3,neg_mean_gamma_deviance,0,0.761812,0.715544,0.671736,0.646801,0.738678
5,2,neg_mean_poisson_deviance,5,0.747300,0.705499,0.652633,0.586905,0.726399
3,2,neg_mean_absolute_error,5,0.733643,0.719091,0.635031,0.581914,0.726367
4,3,neg_mean_poisson_deviance,0,0.775516,0.671427,0.689196,0.600664,0.723471
1,2,r2,5,0.780698,0.662840,0.696317,0.598026,0.721769


In [10]:
fd.to_csv('activity sorting results.csv', index=False) #change output file name